# VIA Soundcloud API

In [1]:
import soundcloud
import json
import requests
output_loc = '/home/will/Documents/Projects/Pi-Music-Server/music_backend/endpoints/res/wills_likes.json'
user_id='79333503'

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
SOUNDCLOUD_KEY = os.getenv("SOUNDCLOUD_ID")

# VIA Selenium Webscraping

In [19]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import time
import clipboard
import requests
import re
page_load_wait_seconds = 3

In [9]:
options = Options()
options.headless = False
browser = webdriver.Firefox(options=options)
browser.get('https://soundcloud.com/you/likes')
err_str = "This error is on purpose. Please log into Soundcloud on the browser, then scroll down and load every liked song into page view. Once you've done this, run the rest of the cells,"
print(err_str)
return None

This error is on purpose. Please log into Soundcloud on the browser, then scroll down and load every liked song into page view. Once you've done this, run the rest of the cells,


SyntaxError: 'return' outside function (<ipython-input-9-eb64db0536c6>, line 7)

### Login to Soundcloud Manually & load entire likes page

In [10]:
song_link_class_name = 'audibleTile__artworkLink'
song_link_elements = browser.find_elements_by_class_name(song_link_class_name)
song_links = list(map(lambda el : el.get_attribute('href'), song_link_elements))

In [11]:
num_song_links = len(song_links)
print(f'{num_song_links} song links found')
for song_link in song_links:
    print(song_link)
print(f'{num_song_links} song links found')

450 song links found
https://soundcloud.com/kotathefriend/mi-casa
https://soundcloud.com/lijofficial/blossom-remix
https://soundcloud.com/whethan/perfectplaces
https://soundcloud.com/yngmartyr/nike-ticks
https://soundcloud.com/7715official/gone-feat-lil-aaron
https://soundcloud.com/sirnijelhimself/cheesecake-cookies
https://soundcloud.com/double_a-ron/lost-count
https://soundcloud.com/hobo_johnson/mover-awayer-5
https://soundcloud.com/alecking13/spotify
https://soundcloud.com/jackharlow/i-wanna-see-some-ass-feat-1
https://soundcloud.com/superduperkylemusic/yes
https://soundcloud.com/rexxliferaj/rexx-life-raj-kenny-beats-moonwalk
https://soundcloud.com/bishopnehru/in-my-zone
https://soundcloud.com/lijofficial/ramen-cups
https://soundcloud.com/lijofficial/jet-plane
https://soundcloud.com/jayidk1/in-my-white-tee
https://soundcloud.com/user-808897583/no-opptions
https://soundcloud.com/y2k2y/go-dumb-feat-bankrol-hayden
https://soundcloud.com/alecking13/vinny-chase
https://soundcloud.com/mat

In [12]:
browser.quit()

In [17]:
# set to false to reset entire library, or set to true to load up to last liked song
# setting it to true saves API hits, only set to false when you want to clear out songs you unliked.
stop_loading_when_detect_repeats = True
prev_song = None
old_data = None
if stop_loading_when_detect_repeats:
    with open(output_loc) as json_file:
        data = json.load(json_file)
        prev_song = data[0]['url']
        old_data = data

In [21]:
wills_songs = []
for i, song_link in enumerate(song_links):
    if song_link == prev_song:
        print(f"breaking at {song_link}, already discovered)
        break
    url = f'https://soundcloud.com/oembed?url={song_link}&client_id={SOUNDCLOUD_KEY}'
    result = str(requests.get(url).content)
    def getValueInTag(tag):
        return result.split(f'<{tag}>')[1].split(f'</{tag}>')[0]
    song_and_artist = getValueInTag('title').split(' by ')
    result = {
        'platform': 'soundcloud',
        'url': song_link,
        'title': song_and_artist[0],
        'artist': song_and_artist[1],
        'artwork_url': getValueInTag('thumbnail-url'),
    }
    # deleteing specified strings
    for attr in ['title', 'artist']:
        to_deletes = ["\\"]
        for to_delete in to_deletes:
            result[attr] = result[attr].replace(to_delete, "")
        # deleteing character codes: format of &#{n many numbers}; (ex: '&#10225;')
        result[attr] = re.sub('\&\#\d*;', '', result[attr])
    
    print(f'{i+1}/{num_song_links}: Added {result["title"]} by {result["artist"]}')
    wills_songs.append(result)
    #print(result)
    time.sleep(0.5) # give the api a little break

SyntaxError: EOL while scanning string literal (<ipython-input-21-bde124aa04bf>, line 4)

In [22]:
print(f'{len(wills_songs)} songs loaded')

8 songs loaded


In [24]:
with open(output_loc, 'w') as f:
    json.dump(wills_songs if not stop_loading_when_detect_repeats else wills_songs + old_data)

TypeError: dump() missing 1 required positional argument: 'fp'